In [1]:
from custom_dataset.segdataset import InstanceSegmentationDataset
from torch.utils.data import DataLoader
import json

import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
import evaluate
import logging
from utils import rs_utils
from datetime import datetime
from transformers import SegformerImageProcessor

In [2]:
#--- root 
root_dir = '/disk3/eric/dataset/Weapon_System/segmentation_pipes'

#--- time
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

In [3]:
#-----------
_VERSION = "090"
_EPOCHS = 20
_MODEL_SAVE = True
_MODEL_VERSION = "nvidia/mit-b5"
_MODEL_VERSION_SAVE = _MODEL_VERSION.split("/")[-1]
_BATCH_SIZE = 2
_DEVICE = "cuda:1"
#----------

In [4]:
# Set up a logger
log_dir = "/home/eric/srcs/FewShotSeg_Lab/FewShotVision_Lab/Segmentation_Pipes/logs"
os.makedirs(log_dir, exist_ok=True)
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
log_file_path = os.path.join(log_dir, f"Version_{_VERSION}_{_MODEL_VERSION_SAVE}_{formatted_time}.log")
logger = rs_utils.setup_logger("FewShotSeg", log_file_path, level=logging.INFO)

In [5]:
json_file = '/disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/_annotations.coco.json'
with open(json_file, 'r') as f:
    data = json.load(f)
#--- 
id2label = {
    0:data['categories'][0]['name'],
    1:data['categories'][1]['name'],
    2:data['categories'][2]['name'],
    3:data['categories'][3]['name'],
    4:data['categories'][4]['name']
}
id2label[0] = "background"
label2id = {v: k for k, v in id2label.items()}

In [6]:
label2id

{'background': 0,
 'M2A1Slammer': 1,
 'M5SandstormMLRS': 2,
 'T140Angara': 3,
 'ZamakMRL': 4}

In [7]:
# for fewshot
id2label = {
    0:"background",
    1:"object"
}
label2id = {v: k for k, v in id2label.items()}

In [8]:
#----------------------
from custom_dataset.segdataset import FewShotSegDataset


image_processor = SegformerImageProcessor(reduce_labels=False,size={"height": 448, "width": 448})

train_dataset = FewShotSegDataset(root_dir=root_dir, image_processor=image_processor)
valid_dataset = FewShotSegDataset(root_dir=root_dir, image_processor=image_processor, train=False)

#train_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor)
#valid_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor, train=False)

train_dataloader = DataLoader(train_dataset, batch_size=_BATCH_SIZE, collate_fn=train_dataset.my_collate_fn, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=_BATCH_SIZE, collate_fn=train_dataset.my_collate_fn)

[('Zamak__part_21.png', 'city_part_4.png'), ('Zamak__part_21.png', 'city_part_9.png'), ('Zamak__part_21.png', 'city_part_13.png'), ('Zamak__part_21.png', 'city_part_10.png'), ('Zamak__part_21.png', 'city_part_24.png'), ('Zamak__part_21.png', 'city_part_3.png'), ('Zamak__part_21.png', 'city_part_19.png'), ('M2A1Slammer2__part_24.png', 'city_part_4.png'), ('M2A1Slammer2__part_24.png', 'city_part_9.png'), ('M2A1Slammer2__part_24.png', 'city_part_13.png'), ('M2A1Slammer2__part_24.png', 'city_part_10.png'), ('M2A1Slammer2__part_24.png', 'city_part_24.png'), ('M2A1Slammer2__part_24.png', 'city_part_3.png'), ('M2A1Slammer2__part_24.png', 'city_part_19.png'), ('Zamak__part_18.png', 'city_part_4.png'), ('Zamak__part_18.png', 'city_part_9.png'), ('Zamak__part_18.png', 'city_part_13.png'), ('Zamak__part_18.png', 'city_part_10.png'), ('Zamak__part_18.png', 'city_part_24.png'), ('Zamak__part_18.png', 'city_part_3.png'), ('Zamak__part_18.png', 'city_part_19.png'), ('M5Sandstorm__part_17.png', 'city_

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [9]:
# # 데이터 배치 확인
# for batch in train_dataloader:
#     print(batch["support"]["images"].shape)  # torch.Size([4, 10])
#     print(batch["query"]["labels"].shape)   # torch.Size([4, 15])

In [10]:
# define model
from models.custom_segformer import FewShotFormer

seg_model = FewShotFormer.from_pretrained(_MODEL_VERSION,
                                        num_labels=2,
                                        id2label=id2label,
                                        label2id=label2id,
                                        force_download=False                                       
                                        )

# model freeze

# model freeze 
for param in seg_model.parameters():
    param.requires_grad = False

# decoder unfreeze 
for param in seg_model.decode_head.parameters():
    param.requires_grad = True

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of FewShotFormer were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_hea

In [11]:
BACKBONE_SIZE = "large" # in ("small", "base", "large" or "giant")

backbone_archs = {
    "small": "vits14",
    "base": "vitb14",
    "large": "vitl14",
    "giant": "vitg14",
}
backbone_arch = backbone_archs[BACKBONE_SIZE]
backbone_name = f"dinov2_{backbone_arch}"

dinov2_model = torch.hub.load(repo_or_dir="facebookresearch/dinov2", model=backbone_name)
#backbone_model.to(_DEVICE)

# model freeze 
for param in dinov2_model.parameters():
    param.requires_grad = False

Using cache found in /home/eric/.cache/torch/hub/facebookresearch_dinov2_main
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [12]:
# define metric
metric = evaluate.load("mean_iou")
# define optimizer
optimizer = torch.optim.AdamW(seg_model.parameters(), lr=0.00006)

In [13]:
best_val_iou = 0 


#-- model to device
seg_model = seg_model.to(_DEVICE)
dinov2_model = dinov2_model.to(_DEVICE) 

#---
for epoch in range(_EPOCHS):  # loop over the dataset multiple times
    logger.info(f"Epoch: {epoch}")

    # Training loop
    for idx, batch in enumerate(train_dataloader):
    #   print(batch["support"]["images"].shape)  # torch.Size([4, 10])
    #   print(batch["query"]["labels"].shape)   # torch.Size([4, 15])        
        #--- support & Query
        support_pixel_values, support_labels = batch["support"]['images'].to(_DEVICE), batch["support"]["labels"].to(_DEVICE) 
        query_pixel_values, query_labels = batch["query"]['images'].to(_DEVICE), batch["query"]["labels"].to(_DEVICE) 
        labels = query_labels
        
        #-----
        patch_token = dinov2_model.forward_features(query_pixel_values)
        patch_token = patch_token["x_norm_patchtokens"]
        patch_token = patch_token.reshape([query_pixel_values.shape[0],-1,32,32])

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = seg_model(pixel_values=support_pixel_values, labels=labels,dino_features = patch_token)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        # Evaluate periodically
        if idx % 100 == 0:
            with torch.no_grad():
                upsampled_logits = nn.functional.interpolate(
                    logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
                )
                predicted = upsampled_logits.argmax(dim=1)

                metrics = metric._compute(
                    predictions=predicted.cpu(),
                    references=labels.cpu(),
                    num_labels=len(id2label),
                    ignore_index=255,
                    reduce_labels=False,  # we've already reduced the labels ourselves
                )
                logger.info(
                    f"Epoch: {epoch}, "
                    f"Training Loss: {loss.item():.4f}, "
                    f"Mean IoU: {metrics['mean_iou']:.4f}, "
                    f"Mean Accuracy: {metrics['mean_accuracy']:.4f}"
                )
        #--
    # Validation loop
    val_loss = 0
    val_metrics = {"mean_iou": 0, "mean_accuracy": 0}
    for batch in valid_dataloader:
        #--- support & Query
        support_pixel_values, support_labels = batch["support"]['images'].to(_DEVICE), batch["support"]["labels"].to(_DEVICE) 
        query_pixel_values, query_labels = batch["query"]['images'].to(_DEVICE), batch["query"]["labels"].to(_DEVICE) 
        labels = query_labels

        with torch.no_grad():
            #-----
            patch_token = dinov2_model.forward_features(query_pixel_values)
            patch_token = patch_token["x_norm_patchtokens"]
            patch_token = patch_token.reshape([query_pixel_values.shape[0],-1,32,32])

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = seg_model(pixel_values=support_pixel_values, labels=labels,dino_features = patch_token)
            loss, logits = outputs.loss, outputs.logits
                
            #--------------            
            val_loss += outputs.loss.item()

            upsampled_logits = nn.functional.interpolate(
                logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            predicted = upsampled_logits.argmax(dim=1)

            batch_metrics = metric._compute(
                predictions=predicted.cpu(),
                references=labels.cpu(),
                num_labels=len(id2label),
                ignore_index=255,
                reduce_labels=False,
            )
            val_metrics["mean_iou"] += batch_metrics["mean_iou"]
            val_metrics["mean_accuracy"] += batch_metrics["mean_accuracy"]

    # Average validation loss and metrics
    val_loss /= len(valid_dataloader)
    val_metrics["mean_iou"] /= len(valid_dataloader)
    val_metrics["mean_accuracy"] /= len(valid_dataloader)

    logger.info(
        f"Epoch : {epoch}, "
        f"Validation Results - Loss: {val_loss:.4f}, "
        f"Mean IoU: {val_metrics['mean_iou']:.4f}, "
        f"Mean Accuracy: {val_metrics['mean_accuracy']:.4f}"
    )

    # Save the model if the validation IoU improves
    if val_metrics["mean_iou"] > best_val_iou:
        best_val_iou = val_metrics["mean_iou"]
        
        if _MODEL_SAVE:
            torch.save(
                seg_model.state_dict(),
                os.path.join(
                    "/disk3/eric/checkpoints/military_fewshot_seg",
                    f"{_VERSION}_{_MODEL_VERSION_SAVE}_fewshotformer_best_epoch_{epoch}_miou_{best_val_iou:.4f}.pt",
                ),
            )
            logger.info("Model saved!")


2024-11-26 22:43:44 - FewShotSeg - INFO - Epoch: 0


filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/Zamak__part_18.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/Zamak__part_18.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_4.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_4.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/T140Angara__part_8.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/T140Angara__part_8.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_4.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_4.png


2024-11-26 22:43:45 - FewShotSeg - INFO - Epoch: 0, Training Loss: 0.6762, Mean IoU: 0.3292, Mean Accuracy: 0.4121


support_states  torch.Size([2, 768, 112, 112])
query_states  torch.Size([2, 1024, 32, 32])
query_states  torch.Size([2, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([2, 448, 448])
weighted_support_states torch.Size([2, 768, 112, 112])
all_factors :  torch.Size([2, 2304, 112, 112])
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/Zamak__part_18.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/Zamak__part_18.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_9.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_9.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/T140Angara__part_8.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/T140Angara__part_8.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_19.png
filename /disk3/eric/dataset

/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


...

support_states  torch.Size([2, 768, 112, 112])
query_states  torch.Size([2, 1024, 32, 32])
query_states  torch.Size([2, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([2, 448, 448])
weighted_support_states torch.Size([2, 768, 112, 112])
all_factors :  torch.Size([2, 2304, 112, 112])
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/images/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/labels/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/images/city_part_15.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/labels/city_part_15.png
support_states  torch.Size([1, 768, 112, 112])
query_states  torch.Size([1, 1024, 32, 32])
query_states  torch.Size([1, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([1, 448, 448])
weighted_support_states torch.Size([1, 768, 112, 112])
all_factors :  torch.Size([1, 230

2024-11-26 22:47:40 - FewShotSeg - INFO - Epoch: 19, Training Loss: 0.0042, Mean IoU: 0.9618, Mean Accuracy: 0.9966


filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/M5Sandstorm__part_17.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/M5Sandstorm__part_17.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_4.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_4.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/M2A1Slammer2__part_11.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/M2A1Slammer2__part_11.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/images/city_part_19.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_19.png
support_states  torch.Size([2, 768, 112, 112])
query_states  torch.Size([2, 1024, 32, 32])
query_states  torch.Size([2, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([2, 448, 448])
weighted_support_sta

2024-11-26 22:47:52 - FewShotSeg - INFO - Epoch : 19, Validation Results - Loss: 0.0042, Mean IoU: 0.9733, Mean Accuracy: 0.9945


support_states  torch.Size([2, 768, 112, 112])
query_states  torch.Size([2, 1024, 32, 32])
query_states  torch.Size([2, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([2, 448, 448])
weighted_support_states torch.Size([2, 768, 112, 112])
all_factors :  torch.Size([2, 2304, 112, 112])
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/images/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/labels/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/images/city_part_15.png
filename /disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/labels/city_part_15.png
support_states  torch.Size([1, 768, 112, 112])
query_states  torch.Size([1, 1024, 32, 32])
query_states  torch.Size([1, 768, 112, 112])
mask type :  <class 'torch.Tensor'>
mask shape :  torch.Size([1, 448, 448])
weighted_support_states torch.Size([1, 768, 112, 112])
all_factors :  torch.Size([1, 230

In [14]:
import torch.nn.functional as F
supp_feat = torch.rand([2, 768, 112, 112])
mask = torch.rand([2, 768, 112, 112])


avg_pool2d = F.avg_pool2d(mask, (supp_feat.size()[2], supp_feat.size()[3])) 

In [15]:
feat_h, feat_w = supp_feat.shape[-2:][0], supp_feat.shape[-2:][1]

In [16]:
feat_h

112

In [17]:
area = avg_pool2d * feat_h * feat_w

In [18]:
area.shape

torch.Size([2, 768, 1, 1])

In [19]:
area.shape

torch.Size([2, 768, 1, 1])

In [20]:
supp_feat.shape[-2:]

torch.Size([112, 112])